In [ ]:
import sqlite3
import time
import datetime

In [ ]:
DB_NAME = "db.sqlite"
file_a = "log_a.txt"
file_b = "log_b.txt"

conn = sqlite3.connect(DB_NAME)

conn.execute("""
CREATE TABLE IF NOT EXISTS logs (
  time DATETIME,
  ticker TEXT
  )
""")
conn.close()

In [ ]:
def write_logfile_to_db(logfile):
    with open(logfile, "r") as f:
        conn = sqlite3.connect(DB_NAME)

        for log in f.readlines():
            _log = log.strip()
            cur = conn.cursor()
            cur.execute('INSERT INTO logs VALUES (?,?)', log.split('|'))

        conn.commit()
        conn.close()

In [ ]:
def clear_logfile(logfile):
    with open(logfile, "w+") as f:
        f.write("")

In [ ]:
def wait_for_next_minute():
    secs = datetime.datetime.utcnow().second
    while secs < 57:
        time.sleep(57 - secs)
        secs = datetime.datetime.utcnow().second
    while datetime.datetime.utcnow().second >= 57:
        time.sleep(1)

In [ ]:
while True:
    
    secs = datetime.datetime.utcnow().second
    
    # if it's a long way to the next minute, sleep for a while
    while secs < 57:
        time.sleep(57 - secs)
        secs = datetime.datetime.utcnow().second
    
    # if it's close to the next minute, write and clear the logs
    while datetime.datetime.utcnow().second >= 57:
    
        if datetime.datetime.now().minute % 2 == 0:
            write_logfile_to_db(file_b)
            clear_logfile(file_b)
            wait_for_next_minute()

        else:
            write_logfile_to_db(file_a)
            clear_logfile(file_a)
            wait_for_next_minute()